In [51]:
include("../src/Julia.jl")

pcgWorker (generic function with 1 method)

In [79]:
gname = "../../graphs/grid3_iu_27000_1/"
wmax = 1000
iu = true
n = 30;

In [80]:
function wgen()
    if iu
        return 1 / rand(1:wmax)
    else
        return rand(1:wmax)
    end
end

wgen (generic function with 1 method)

In [81]:
a = wGrid3(n, weightGen=wgen)
a.n,ceil(Int64,nnz(a)/2)

(27000,78300)

In [82]:
tree1 = akpw(a);
str1 = compStretches(tree1,a);
data1 = sum(str1.nzval)/2,mean(str1.nzval)

(565359.0249672276,7.22042177480495)

In [83]:
tree2 = randishPrim(a);
str2 = compStretches(tree2,a);
data2 = sum(str2.nzval)/2,mean(str2.nzval)

(1.5427902389283578e6,19.703579041230622)

In [84]:
tree3 = randishKruskal(a);
str3 = compStretches(tree3,a);
data3 = sum(str3.nzval)/2,mean(str3.nzval)

(2.1693118733057408e6,27.7051324815548)

In [85]:
truex = randn(n * n * n); truex = truex - mean(truex);
b = lap(a) * truex; b = b - mean(b);

In [86]:
writeToFile(gname * "graph.mtx", a);

In [87]:
writeToFile(gname * "tree1.mtx", tree1);
writeToFile(gname * "tree2.mtx", tree2);
writeToFile(gname * "tree3.mtx", tree3);

In [88]:
writeToFile(gname * "x.vec", truex);
writeToFile(gname * "b.vec", b);

In [89]:
function strDist{Tv,Ti}(tr::SparseMatrixCSC{Tv,Ti}, a::SparseMatrixCSC{Tv,Ti})
    
    res = Array{ASCIIString,1}(0)
    
    str = compStretches(tr, a-tr).nzval;
    
    vmax = maximum(str)
    inter = (0,1)
    while inter[2] < vmax
        cnt = 0
        for i in 1:length(str)
            if inter[1] <= str[i] && str[i] < inter[2]
                cnt = cnt + 1
            end
        end
        # don't count edges twice
        cnt = ceil(Int64,cnt / 2)
        
        push!(res, "($(inter[1]),$(inter[2])) - $(cnt)")
        
        inter = (inter[2], inter[2] * 2)
    end
    
    push!(res, "")
    
    return res;
    
end

strDist (generic function with 1 method)

In [90]:
output = []
push!(output, "A grid3 with $(n^3) vertices and $(ceil(Int64,nnz(a)/2)) edges.")
if iu
    push!(output, "Weights are reals sampled uniformly from 1/[1..$(wmax)]")
else
    push!(output, "Weights are integers sampled uniformly from [1..$(wmax)]")
end
push!(output, "")
push!(output, "There are three trees in the folder.")
push!(output, "")
push!(output, "Tree1 - total stretch = $(data1[1]), ave stretch = $(data1[2])")
append!(output, strDist(tree1,a))
push!(output, "Tree2 - total stretch = $(data2[1]), ave stretch = $(data2[2])")
append!(output, strDist(tree2,a))
push!(output, "Tree3 - total stretch = $(data3[1]), ave stretch = $(data3[2])")
append!(output, strDist(tree3,a))

desc = open(gname * "description.txt", "w")
for i in 1:length(output)
    println(desc, output[i])
end
close(desc)